# Video Processing Exploration 

## Project: Smart Media Analyzer (Frugal Architecture)

### Goals
- Test video scene detection with PySceneDetect
- Document timing and accuracy for different video types

### Current Stack
- **Python:** 3.11.13 (UV managed)
- **Scene Detection:** PySceneDetect 0.6.6
- **Video Processing:** OpenCV 4.11.0

### Test Videos
- `Giant_Oarfish.mp4` - Nature documentary (~68 seconds)
- More videos to be added...

---

### Video Processing Setup - Testing imports and timing

In [ ]:
#!/usr/bin/env python3
"""
Video Processing Setup - Testing imports and timing
"""
import time
import os
from datetime import datetime
from datetime import datetime

def log_time(message):
    """Print message with timestamp for performance tracking"""
    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    print(f"[{timestamp}] {message}")

# Test imports with timing
log_time("=== Starting Video Processing Setup ===")

log_time("Importing cv2...")
start = time.time()
import cv2
log_time(f"cv2 imported in {time.time() - start:.3f}s - Version: {cv2.__version__}")

log_time("Importing scenedetect...")
start = time.time()
from scenedetect import detect, ContentDetector
log_time(f"scenedetect imported in {time.time() - start:.3f}s")

log_time("All imports successful!")

### Analyzing video file using OpenCV module

In [ ]:
def analyze_video_file(video_path):
    """Get detailed information about a video file"""
    log_time(f"Analyzing video file: {video_path}")
    
    # Check if file exists
    if not os.path.exists(video_path):
        print(f"ERROR: Video file not found: {video_path}")
        return None
    
    file_size_mb = os.path.getsize(video_path) / (1024 * 1024)
    log_time(f"File exists - Size: {file_size_mb:.2f} MB")
    
    # Get video properties using OpenCV
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("ERROR: Cannot open video with OpenCV")
        return None
    
    # Extract video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps if fps > 0 else 0
    
    cap.release()
    
    # Display results
    print(f"Current Time: {datetime.now()}")
    print(f" Video Properties:")
    print(f"   Resolution: {width} x {height}")
    print(f"   FPS: {fps:.2f}")
    print(f"   Duration: {duration:.2f} seconds")
    print(f"   Total Frames: {frame_count}")
    print(f"   File Size: {file_size_mb:.2f} MB")
    
    return {
        'width': width, 'height': height, 'fps': fps,
        'duration': duration, 'frames': frame_count, 'size_mb': file_size_mb
    }

# Test with your Giant Oarfish video
video_file = "../Samples_Video-Images/Giant_Oarfish.mp4"
print(f"Analyzing video file: {video_file}")
video_info = analyze_video_file(video_file)

### Scene Detection Analysis

In [ ]:
def detect_scenes_with_analysis(video_path, threshold=27.0):
    """
    Detect scenes and provide detailed analysis
    """
    log_time(f"Starting scene detection (threshold={threshold})")
    
    start_time = time.time()
    
    # Detect scenes using ContentDetector
    scene_list = detect(video_path, ContentDetector(threshold=threshold))
    
    detection_time = time.time() - start_time
    log_time(f"Scene detection completed in {detection_time:.2f}s")
    
    # Analysis
    total_scenes = len(scene_list)
    log_time(f"Found {total_scenes} scenes")
    
    if total_scenes == 0:
        print("⚠️  No scenes detected - video might be too uniform")
        return scene_list
    
    # Calculate scene statistics
    durations = []
    print(f"\n📊 Scene Analysis:")
    print(f"{'Scene':<8} {'Start':<8} {'End':<8} {'Duration':<10}")
    print("-" * 35)
    
    for i, scene in enumerate(scene_list):
        start_sec = scene[0].get_seconds()
        end_sec = scene[1].get_seconds()
        duration = end_sec - start_sec
        durations.append(duration)
        
        print(f"{i+1:<8} {start_sec:<8.2f} {end_sec:<8.2f} {duration:<10.2f}")
    
    # Statistics
    avg_duration = sum(durations) / len(durations)
    min_duration = min(durations)
    max_duration = max(durations)
    
    print(f"\n📈 Scene Statistics:")
    print(f"   Total scenes: {total_scenes}")
    print(f"   Average duration: {avg_duration:.2f}s")
    print(f"   Shortest scene: {min_duration:.2f}s")
    print(f"   Longest scene: {max_duration:.2f}s")
    print(f"   Processing speed: {67.86/detection_time:.1f}x real-time")
    
    return scene_list

# Run scene detection on your video
scenes = detect_scenes_with_analysis(video_file)

### Threshold Experimentation (Code Cell)

In [ ]:
def test_multiple_videos_with_thresholds():
    """
    Test scene detection with multiple videos and different thresholds
    """
    log_time("=== Testing Multiple Video Files with Threshold Analysis ===")
    
    # List your available video files here
    video_files = [
        "../Samples_Video-Images/Giant_Oarfish.mp4",
        "../Samples_Video-Images/SoccorShootout_1.mp4"
    ]
    
    # Check which files exist
    available_videos = []
    for video_path in video_files:
        if os.path.exists(video_path):
            available_videos.append(video_path)
            print(f"✅ Found: {video_path}")
        else:
            print(f"❌ Missing: {video_path}")
    
    print(f"\n{'='*60}")
    
    # Test each video with different thresholds
    thresholds = [15.0, 20.0, 27.0, 35.0, 45.0]
    all_results = {}
    
    for video_path in available_videos:
        print(f"\n🎬 Analyzing: {os.path.basename(video_path)}")
        
        # Get video info first
        video_info = analyze_video_file(video_path)
        if not video_info:
            continue
            
        print(f"\n📊 Threshold Analysis for {os.path.basename(video_path)}:")
        print(f"{'Threshold':<12} {'Scenes':<8} {'Avg Duration':<12} {'Time':<8}")
        print("-" * 45)
        
        video_results = []
        
        for threshold in thresholds:
            start_time = time.time()
            scene_list = detect(video_path, ContentDetector(threshold=threshold))
            detection_time = time.time() - start_time
            
            scene_count = len(scene_list)
            avg_duration = video_info['duration'] / scene_count if scene_count > 0 else 0
            
            video_results.append({
                'threshold': threshold,
                'scenes': scene_count,
                'avg_duration': avg_duration,
                'time': detection_time
            })
            
            print(f"{threshold:<12} {scene_count:<8} {avg_duration:<12.2f} {detection_time:<8.2f}s")
        
        all_results[video_path] = {
            'info': video_info,
            'threshold_results': video_results
        }
        
        print(f"\n{'='*60}")
    
    # Comparison summary
    if len(available_videos) > 1:
        print(f"\n🔍 Video Comparison Summary (threshold=27.0):")
        print(f"{'Video':<25} {'Duration':<10} {'Scenes':<8} {'Avg Scene':<10} {'Type':<15}")
        print("-" * 75)
        
        for video_path in available_videos:
            if video_path in all_results:
                info = all_results[video_path]['info']
                # Find threshold=27.0 result
                result_27 = next(r for r in all_results[video_path]['threshold_results'] if r['threshold'] == 27.0)
                
                video_name = os.path.basename(video_path)[:20]
                video_type = "Nature Doc" if "oarfish" in video_path.lower() else "Sports" if "soccor" in video_path.lower() else "Unknown"
                
                print(f"{video_name:<25} {info['duration']:<10.1f} {result_27['scenes']:<8} {result_27['avg_duration']:<10.2f} {video_type:<15}")
    
    return all_results

# Run multi-video threshold analysis
multi_video_results = test_multiple_videos_with_thresholds()

### Frame Extraction Function
Extract the frames from the above scene detection logic.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Configure matplotlib for better display
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams['figure.dpi'] = 100

def extract_keyframes_from_scenes(video_path, scene_list, frames_per_scene=1, frame_position='middle'):
    """
    Extract keyframes from detected scenes
    
    Args:
        video_path: Path to video file
        scene_list: List of scenes from PySceneDetect
        frames_per_scene: Number of frames to extract per scene (1, 2, or 3)
        frame_position: 'start', 'middle', 'end', or 'all'
    
    Returns:
        List of frame data with metadata
    """
    log_time(f"Extracting keyframes from {len(scene_list)} scenes...")
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Cannot open video: {video_path}")
        return []
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames_extracted = 0
    extracted_frames = []
    
    for scene_idx, scene in enumerate(scene_list):
        scene_start_sec = scene[0].get_seconds()
        scene_end_sec = scene[1].get_seconds()
        scene_duration = scene_end_sec - scene_start_sec
        
        # Calculate frame positions based on preference
        frame_times = []
        
        if frame_position == 'start':
            frame_times = [scene_start_sec + 0.1]  # 0.1s after start
        elif frame_position == 'end':
            frame_times = [scene_end_sec - 0.1]    # 0.1s before end
        elif frame_position == 'middle':
            frame_times = [scene_start_sec + (scene_duration / 2)]
        elif frame_position == 'all' and frames_per_scene == 3:
            frame_times = [
                scene_start_sec + (scene_duration * 0.2),  # 20% into scene
                scene_start_sec + (scene_duration * 0.5),  # 50% into scene  
                scene_start_sec + (scene_duration * 0.8)   # 80% into scene
            ]
        
        # Extract frames at calculated times
        for frame_idx, frame_time in enumerate(frame_times):
            frame_number = int(frame_time * fps)
            
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            ret, frame = cap.read()
            
            if ret:
                # Convert BGR to RGB for matplotlib
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                extracted_frames.append({
                    'scene_number': scene_idx + 1,
                    'frame_index': frame_idx + 1,
                    'timestamp': frame_time,
                    'scene_start': scene_start_sec,
                    'scene_end': scene_end_sec,
                    'scene_duration': scene_duration,
                    'frame_number': frame_number,
                    'frame_data': frame_rgb,
                    'frame_shape': frame_rgb.shape
                })
                
                total_frames_extracted += 1
    
    cap.release()
    log_time(f"✅ Extracted {total_frames_extracted} keyframes from {len(scene_list)} scenes")
    
    return extracted_frames

def visualize_extracted_keyframes(extracted_frames, max_display=10):
    """
    Display extracted keyframes in a grid
    """
    if not extracted_frames:
        print("❌ No frames to display")
        return
    
    frames_to_show = min(max_display, len(extracted_frames))
    log_time(f"Displaying first {frames_to_show} keyframes...")
    
    # Calculate grid dimensions
    cols = 3 if frames_to_show > 6 else 2 if frames_to_show > 2 else 1
    rows = (frames_to_show + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    if rows == 1 and cols == 1:
        axes = [axes]
    elif rows == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    for i in range(frames_to_show):
        frame_info = extracted_frames[i]
        
        axes[i].imshow(frame_info['frame_data'])
        axes[i].set_title(
            f"Scene {frame_info['scene_number']} - {frame_info['timestamp']:.2f}s\n"
            f"Duration: {frame_info['scene_duration']:.2f}s", 
            fontsize=10
        )
        axes[i].axis('off')
    
    # Hide empty subplots
    for i in range(frames_to_show, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

def save_keyframes_metadata(extracted_frames, output_file="keyframes_metadata.json"):
    """
    Save keyframe metadata to JSON file (without actual image data)
    """
    import json
    
    metadata = []
    for frame_info in extracted_frames:
        metadata.append({
            'scene_number': frame_info['scene_number'],
            'frame_index': frame_info['frame_index'],
            'timestamp': frame_info['timestamp'],
            'scene_start': frame_info['scene_start'],
            'scene_end': frame_info['scene_end'],
            'scene_duration': frame_info['scene_duration'],
            'frame_number': frame_info['frame_number'],
            'frame_shape': frame_info['frame_shape']
        })
    
    with open(output_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    log_time(f"📄 Saved metadata for {len(metadata)} frames to {output_file}")

# Test frame extraction with your video
log_time("=== Testing Frame Extraction ===")

# Extract middle frame from each scene for Giant Oarfish video
oarfish_frames = extract_keyframes_from_scenes(
    "../Samples_Video-Images/Giant_Oarfish.mp4", 
    scenes,  # Using scenes from previous cell
    frames_per_scene=1,
    frame_position='middle'
)

# Display the keyframes
visualize_extracted_keyframes(oarfish_frames, max_display=3)

# Save metadata
save_keyframes_metadata(oarfish_frames, "giant_oarfish_keyframes.json")

### OpenCLIP Exploration
https://openai.com/index/clip/
#### Test OpenCLIP installation

In [ ]:
# Test OpenCLIP installation
try:
    import open_clip
    import torch
    
    print(f"✅ OpenCLIP imported successfully")
    print(f"✅ PyTorch version: {torch.__version__}")
    print(f"✅ MPS (M1 GPU) available: {torch.backends.mps.is_available()}")
    
    # List available models
    available_models = open_clip.list_pretrained()
    print(f"✅ Available models: {len(available_models)} models found")
    print("First few models:", available_models[:3])
    
except ImportError as e:
    print(f"❌ Import error: {e}")

#### Basic OpenCLIP; first frame demo

In [ ]:
import open_clip
import torch
from PIL import Image
import requests
import numpy as np

# Load a lightweight model optimized for M1
model, preprocess = open_clip.create_model_from_pretrained(
    'ViT-B-32', 
    pretrained='openai',
    device='mps' if torch.backends.mps.is_available() else 'cpu'
)

tokenizer = open_clip.get_tokenizer('ViT-B-32')

log_time("✅ OpenCLIP model loaded successfully")

# Test with one of your extracted keyframes
def analyze_keyframe_with_openclip(frame_data, text_queries):
    """
    Analyze a keyframe using OpenCLIP
    """
    # Convert your extracted frame to PIL Image
    pil_image = Image.fromarray(frame_data)
    
    # Preprocess image
    image_input = preprocess(pil_image).unsqueeze(0)
    if torch.backends.mps.is_available():
        image_input = image_input.to('mps')
    
    # Tokenize text queries
    text_input = tokenizer(text_queries)
    if torch.backends.mps.is_available():
        text_input = text_input.to('mps')
    
    # Generate embeddings
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_input)
        
        # Calculate similarities
        similarities = (image_features @ text_features.T).softmax(dim=-1)
    
    return similarities.cpu().numpy()[0]

# Test with your first keyframe
if 'oarfish_frames' in globals() and len(oarfish_frames) > 0:
    test_queries = [
        "a large fish swimming underwater",
        "marine life in the ocean", 
        "underwater scene",
        "a person on land",
        "a car driving"
    ]
    
    similarities = analyze_keyframe_with_openclip(
        oarfish_frames[0]['frame_data'], 
        test_queries
    )
    
    print("🎯 Similarity Scores for First Keyframe:")
    for query, score in zip(test_queries, similarities):
        print(f"   {query}: {score:.3f}")
        
    # Find best match
    best_match_idx = np.argmax(similarities)
    print(f"\n🏆 Best match: '{test_queries[best_match_idx]}' (score: {similarities[best_match_idx]:.3f})")

#### Analyze first few frames using OpenCLIP

In [ ]:
# Test with first 5 keyframes
if 'oarfish_frames' in globals() and len(oarfish_frames) > 0:
    test_queries = [
        "a large fish swimming underwater",
        "marine life in the ocean", 
        "underwater scene",
        "a person on land",
        "a car driving"
    ]
    
    total_frames = len(oarfish_frames)
    frames_to_analyze = min(5, total_frames)  # In case there are fewer than 5 frames
    
    print(f"🎯 Analyzing first 5 frames from {total_frames} total frames:")
    print("=" * 60)
    
    for i in range(frames_to_analyze):
        frame_info = oarfish_frames[i]
        
        # Analyze this frame
        similarities = analyze_keyframe_with_openclip(
            frame_info['frame_data'], 
            test_queries
        )
        
        print(f"\n📋 Frame {i+1}/5 - Scene {frame_info['scene_number']} at {frame_info['timestamp']:.2f}s:")
        print("-" * 40)
        
        # Show all similarity scores
        for query, score in zip(test_queries, similarities):
            print(f"   {query}: {score:.3f}")
        
        # Find and highlight best match
        best_match_idx = np.argmax(similarities)
        best_score = similarities[best_match_idx]
        print(f"   🏆 Best: '{test_queries[best_match_idx]}' ({best_score:.3f})")
        
        # Show confidence level
        if best_score > 0.7:
            confidence = "🟢 High confidence"
        elif best_score > 0.5:
            confidence = "🟡 Medium confidence"
        else:
            confidence = "🔴 Low confidence"
        print(f"   {confidence}")

#### OpenCLIP Comprehensive Analysis

Creates category vocabulary - Defines 50+ visual concepts across marine life, environments, objects, actions, and visual properties.

Analyzes keyframes - Compares each frame against all categories using AI vision-language understanding.

Returns structured metadata - Provides confidence scores for detected features (e.g., "large fish: 0.89", "deep sea: 0.78").

Handles edge cases - Uses fallback classification for low-confidence frames to ensure every frame gets some description

In [ ]:
def create_comprehensive_categories():
    """
    Create hierarchical categories covering visual content
    TODO: Expand these category lists for more comprehensive coverage
    """
    categories = {
        # LIVING THINGS
        "marine_animals": [
            "small fish", "large fish", "school of fish", "shark", "whale", 
            "dolphin", "octopus", "jellyfish", "sea turtle", "ray", "eel",
            "marine animal", "sea creature", "underwater animal"
        ],
        
        "land_animals": [
            "bird", "mammal", "reptile", "domestic animal", "wild animal"
        ],
        
        "humans": [
            "person", "people", "diver", "swimmer", "child", "adult"
        ],
        
        # ENVIRONMENTS  
        "water_environments": [
            "ocean", "deep sea", "shallow water", "coral reef", "open water",
            "underwater cave", "clear water", "murky water"
        ],
        
        "land_environments": [
            "beach", "forest", "desert", "mountain", "urban area", "indoor space"
        ],
        
        # OBJECTS & ACTIONS
        "objects": [
            "boat", "ship", "rock", "coral", "seaweed", "equipment"
        ],
        
        "movements": [
            "swimming", "floating", "diving", "resting", "feeding", "hunting"
        ],
        
        # VISUAL PROPERTIES
        "image_composition": [
            "close-up", "wide shot", "medium shot", "underwater view"
        ],
        
        "lighting_conditions": [
            "bright light", "dim light", "underwater lighting", "natural lighting"
        ],
        
        # GENERAL CATEGORIES (fallback)
        "general_content": [
            "natural scene", "calm scene", "interesting content", "documentary style"
        ]
    }
    
    return categories

def analyze_frame_with_fallback(frame_data, confidence_threshold=0.15):
    """
    Comprehensive analysis with fallback for unmatched content
    """
    categories = create_comprehensive_categories()
    
    pil_image = Image.fromarray(frame_data)
    image_input = preprocess(pil_image).unsqueeze(0)
    if torch.backends.mps.is_available():
        image_input = image_input.to('mps')
    
    results = {}
    all_confidences = []
    
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        
        # Analyze each category group
        for category_name, items in categories.items():
            text_input = tokenizer(items)
            if torch.backends.mps.is_available():
                text_input = text_input.to('mps')
                
            text_features = model.encode_text(text_input)
            similarities = (image_features @ text_features.T).softmax(dim=-1)
            
            # Get matches above threshold
            high_confidence_items = []
            for i, score in enumerate(similarities[0]):
                if score > confidence_threshold:
                    high_confidence_items.append({
                        "label": items[i],
                        "confidence": score.item()
                    })
                    all_confidences.append(score.item())
            
            if high_confidence_items:
                # Sort by confidence and keep top 3
                high_confidence_items.sort(key=lambda x: x['confidence'], reverse=True)
                results[category_name] = high_confidence_items[:3]
    
    # FALLBACK for low-confidence frames
    max_confidence = max(all_confidences) if all_confidences else 0
    
    if max_confidence < 0.3:
        fallback_categories = [
            "visual content", "video frame", "general scene", "recorded material"
        ]
        
        text_input = tokenizer(fallback_categories)
        if torch.backends.mps.is_available():
            text_input = text_input.to('mps')
            
        text_features = model.encode_text(text_input)
        similarities = (image_features @ text_features.T).softmax(dim=-1)
        
        best_fallback = similarities[0].argmax()
        results["fallback_classification"] = {
            "label": fallback_categories[best_fallback],
            "confidence": similarities[0][best_fallback].item(),
            "note": "Low confidence - using generic classification"
        }
    
    # Add analysis metadata
    total_categories = sum(len(items) for items in categories.values())
    results["analysis_metadata"] = {
        "total_categories_tested": total_categories,
        "max_confidence": max_confidence,
        "avg_confidence": sum(all_confidences) / len(all_confidences) if all_confidences else 0,
        "features_detected": len(all_confidences),
        "analysis_quality": "high" if max_confidence > 0.7 else "medium" if max_confidence > 0.4 else "low"
    }
    
    return results

# Test with first keyframe
if 'oarfish_frames' in globals() and len(oarfish_frames) > 0:
    log_time("=== Starting Comprehensive OpenCLIP Analysis ===")
    
    # Analyze first frame
    frame_info = oarfish_frames[0]
    result = analyze_frame_with_fallback(
        frame_info['frame_data'], 
        confidence_threshold=0.20
    )
    
    print("🎯 Comprehensive Analysis Results:")
    print("=" * 50)
    
    # DISPLAY FRAME IMAGE FIRST
    print(f"\n🖼️  ANALYZING FRAME:")
    print(f"   Scene {frame_info['scene_number']} at {frame_info['timestamp']:.2f}s")
    
    # Show the image
    plt.figure(figsize=(6, 4))
    plt.imshow(frame_info['frame_data'])
    plt.title(f"Scene {frame_info['scene_number']} - Keyframe at {frame_info['timestamp']:.2f}s", 
              fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Then show analysis results
    for category, items in result.items():
        if category != "analysis_metadata" and category != "fallback_classification":
            print(f"\n📋 {category.upper().replace('_', ' ')}:")
            for item in items:
                confidence_icon = "🟢" if item['confidence'] > 0.7 else "🟡" if item['confidence'] > 0.4 else "🔴"
                print(f"   {confidence_icon} {item['label']}: {item['confidence']:.3f}")
    
    # Show fallback if present
    if "fallback_classification" in result:
        print(f"\n⚠️  FALLBACK:")
        fallback = result["fallback_classification"]
        print(f"   {fallback['label']}: {fallback['confidence']:.3f}")
        print(f"   Note: {fallback['note']}")
    
    # Show metadata
    print(f"\n📊 ANALYSIS SUMMARY:")
    metadata = result["analysis_metadata"]
    print(f"   • Quality: {metadata['analysis_quality'].upper()}")
    print(f"   • Best confidence: {metadata['max_confidence']:.3f}")
    print(f"   • Features detected: {metadata['features_detected']}")
    print(f"   • Categories tested: {metadata['total_categories_tested']}")
    
    log_time("✅ Comprehensive analysis completed")

else:
    print("❌ oarfish_frames not found. Please run frame extraction first.")

#### Analyzing multiple frames, show extarcted output; only show categories where confidence is > 0.7

In [ ]:
# Test with first 5 keyframes
if 'oarfish_frames' in globals() and len(oarfish_frames) > 0:
    log_time("=== Starting Comprehensive OpenCLIP Analysis ===")
    
    frames_to_analyze = min(5, len(oarfish_frames))  # Analyze up to 5 frames
    
    print("🎯 Comprehensive Analysis Results:")
    print("=" * 60)
    
    for frame_idx in range(frames_to_analyze):
        frame_info = oarfish_frames[frame_idx]
        
        # Analyze current frame
        result = analyze_frame_with_fallback(
            frame_info['frame_data'], 
            confidence_threshold=0.20
        )
        
        print(f"\n🖼️  FRAME {frame_idx + 1}/5:")
        print(f"   Scene {frame_info['scene_number']} at {frame_info['timestamp']:.2f}s")
        print(f"   Duration: {frame_info['scene_duration']:.2f}s")
        
        # Show the image using plt.figure
        plt.figure(figsize=(6, 4))
        plt.imshow(frame_info['frame_data'])
        plt.title(f"Frame {frame_idx + 1} - Scene {frame_info['scene_number']} at {frame_info['timestamp']:.2f}s", 
                  fontsize=12, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        # Show analysis results for this frame - ONLY confidence > 0.7
        high_confidence_found = False
        for category, items in result.items():
            if category != "analysis_metadata" and category != "fallback_classification":
                # Filter items with confidence > 0.7
                high_confidence_items = [item for item in items if item['confidence'] > 0.7]
                
                if high_confidence_items:
                    if not high_confidence_found:  # First high confidence category
                        high_confidence_found = True
                    
                    print(f"\n📋 {category.upper().replace('_', ' ')} (High Confidence):")
                    for item in high_confidence_items:
                        print(f"   🟢 {item['label']}: {item['confidence']:.3f}")
        
        # If no high confidence results found, show a message
        if not high_confidence_found:
            print(f"\n⚠️  No high confidence results (>0.7) found for this frame")
            # Show top result from metadata
            metadata = result["analysis_metadata"]
            print(f"   Highest confidence achieved: {metadata['max_confidence']:.3f}")
        
        # Show fallback if present
        if "fallback_classification" in result:
            fallback = result["fallback_classification"]
            if fallback['confidence'] > 0.7:
                print(f"\n⚠️  FALLBACK (High Confidence):")
                print(f"   🟢 {fallback['label']}: {fallback['confidence']:.3f}")
                print(f"   Note: {fallback['note']}")
        
        # Show metadata summary
        metadata = result["analysis_metadata"]
        print(f"\n📊 FRAME {frame_idx + 1} SUMMARY:")
        print(f"   • Quality: {metadata['analysis_quality'].upper()}")
        print(f"   • Best confidence: {metadata['max_confidence']:.3f}")
        print(f"   • High confidence features (>0.7): {sum(1 for category, items in result.items() if category not in ['analysis_metadata', 'fallback_classification'] for item in items if item['confidence'] > 0.7)}")
        
        # Add separator between frames
        if frame_idx < frames_to_analyze - 1:  # Don't add separator after last frame
            print("\n" + "─" * 60)
    
    log_time("✅ Comprehensive analysis of 5 frames completed")

else:
    print("❌ oarfish_frames not found. Please run frame extraction first.")

=============================
### Database Checks for testing

In [ ]:
import sys
import os

# Add parent directory to Python path
sys.path.append(os.path.dirname(os.getcwd()))

# Now import should work
from database import print_dependency_status, check_dependencies

print_dependency_status()
deps = check_dependencies()
print(f"All required deps available: {all(deps.values())}")

### Initialize database

In [ ]:
# Setup (same as before)
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Create database - now will create in database/databases/
from database import MediaArchiveDB
db = MediaArchiveDB()

print(f"✅ Database created at: {db.db_path}")
print(f"📁 Full path: {Path(db.db_path).absolute()}")

# Verify the folder structure
import os
print(f"Database file exists: {os.path.exists(db.db_path)}")

### Test with your existing video data

In [ ]:
# In your notebook, test the new path
success, video_id, error = db.add_video({
    'filepath': '../Samples_Video-Images/Giant_Oarfish.mp4',
    'filename': 'Giant_Oarfish.mp4',
    'duration_seconds': 67.86,
    'fps': 23.98,
    'width': 1280,
    'height': 720,
    'total_frames': 1627,
    'file_size_mb': 11.76
})

print(f"Add video result: Success={success}, ID={video_id}")
print(f"Database location: {db.db_path}")